In [1]:
!pip install gensim
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 5.1 MB/s 
     |████████████████████████████████| 2.2 MB 53.0 MB/s 


In [2]:
import gensim
import pandas as pd

In [3]:
!gdown http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz

Downloading...
From: http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz
To: /content/reviews_Cell_Phones_and_Accessories_5.json.gz
100% 45.4M/45.4M [00:03<00:00, 13.3MB/s]


In [7]:
!ls

reviews_Cell_Phones_and_Accessories_5.json  sample_data


In [6]:
!gunzip -d reviews_Cell_Phones_and_Accessories_5.json.gz

In [8]:
df = pd.read_json("reviews_Cell_Phones_and_Accessories_5.json", lines=True)
df.head(10)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"
5,APX47D16JOP7H,120401325X,RLH,"[1, 2]",These make using the home button easy. My daug...,3,Cute,1381536000,"10 12, 2013"
6,A1JVVYYO7G56DS,120401325X,Tyler Evans,"[0, 0]",Came just as described.. It doesn't come unstu...,5,best thing ever..,1377129600,"08 22, 2013"
7,A6FGO4TBZ3QFZ,3998899561,Abdullah Albyati,"[1, 2]",it worked for the first week then it only char...,1,not a good Idea,1384992000,"11 21, 2013"
8,A2JWEDW5FSVB0F,3998899561,Adam,"[2, 3]","Good case, solid build. Protects phone all aro...",5,Solid Case,1380067200,"09 25, 2013"
9,A8AJS1DW7L3JJ,3998899561,Agata Majchrzak,"[1, 1]",This is a fantastic case. Very stylish and pro...,5,Perfect Case,1396483200,"04 3, 2014"


In [9]:
df.shape

(194439, 9)

In [10]:
#Simple Preprocessing & Tokenization
review_text = df.reviewText.apply(gensim.utils.simple_preprocess)

In [11]:
review_text

0         [they, look, good, and, stick, good, just, don...
1         [these, stickers, work, like, the, review, say...
2         [these, are, awesome, and, make, my, phone, lo...
3         [item, arrived, in, great, time, and, was, in,...
4         [awesome, stays, on, and, looks, great, can, b...
                                ...                        
194434    [works, great, just, like, my, original, one, ...
194435    [great, product, great, packaging, high, quali...
194436    [this, is, great, cable, just, as, good, as, t...
194437    [really, like, it, becasue, it, works, well, w...
194438    [product, as, described, have, wasted, lot, of...
Name: reviewText, Length: 194439, dtype: object

In [12]:
review_text.loc[0]

['they',
 'look',
 'good',
 'and',
 'stick',
 'good',
 'just',
 'don',
 'like',
 'the',
 'rounded',
 'shape',
 'because',
 'was',
 'always',
 'bumping',
 'it',
 'and',
 'siri',
 'kept',
 'popping',
 'up',
 'and',
 'it',
 'was',
 'irritating',
 'just',
 'won',
 'buy',
 'product',
 'like',
 'this',
 'again']

In [13]:
df.reviewText.loc[0]

"They look good and stick good! I just don't like the rounded shape because I was always bumping it and Siri kept popping up and it was irritating. I just won't buy a product like this again"

In [14]:
#Training the Word2Vec Model
#Initialize the model
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4,)

In [15]:
#Build Vocabulary
model.build_vocab(review_text, progress_per=1000)

In [16]:
#Train the Word2Vec Model
model.train(review_text, total_examples=model.corpus_count, epochs=model.epochs)

(61505671, 83868975)

In [17]:
#Finding Similar Words and Similarity between words
model.wv.most_similar("bad")

[('terrible', 0.6638950109481812),
 ('shabby', 0.6094961762428284),
 ('horrible', 0.602645993232727),
 ('good', 0.5931912064552307),
 ('awful', 0.552463173866272),
 ('sad', 0.5490090847015381),
 ('okay', 0.5352621674537659),
 ('funny', 0.5194658041000366),
 ('disappointing', 0.5110157132148743),
 ('cheap', 0.5090533494949341)]

In [18]:
model.wv.similarity(w1="cheap", w2="inexpensive")

0.52406824

In [19]:
model.wv.similarity(w1="great", w2="good")

0.7918587